In [1]:
import torch
import pyro
import pyro.distributions as dist

### Examples: 

##### Unfair coin

##### model:

In [117]:
def coin_model0(guess):
    coin = pyro.sample("coin", dist.Uniform(0, 1))
    print("original coin:", coin)
    # return pyro.sample("flip", dist.Bernoulli(probs=coin))
    # i = 0
    with pyro.plate("data_loop",size=170):
        # i+=1
        result = pyro.sample("flip", dist.Bernoulli(probs=coin))
    # print(i)
    return(result)
    

In [118]:
# data = torch.tensor([coin_model0(1) for i in range(10)])
# data
data = coin_model0(2)
data

original coin: tensor(0.8203)


tensor([1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
        0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 1., 1.])

##### condition:

In [119]:
# data = torch.zeros(10)
# data[0:6] = torch.ones(6)
from torch.distributions import constraints

# condition0
def conditioned_coin_obs(data):
    # print("conditioned guess:",guess)
    
    # bug here with Uniform(0, 1)
    coin = pyro.sample("coin", dist.Uniform(0, 2))
    # print("conditioned: coin: ", coin)
    
    # sample 10 times:
    with pyro.plate("data_loop", size=170):   
         result = pyro.sample("flip", dist.Bernoulli(probs=coin),
                              obs=data)
    # print("conditioned: flip", result)
    return(result)

In [120]:
conditioned_coin_obs(data)

tensor([1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1.,
        0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 1., 1.])

##### guide:

In [121]:
def coin_parametrized_guide(data):
   
    # bug here with interval(0, 1)
    aa = pyro.param("a", torch.tensor(0.4),
                    constraint=constraints.interval(0, 0.9))
    # print("guide a:", aa)

    res = pyro.sample("coin", dist.Bernoulli(probs=aa))
    # print("guide: coin", res)
    return res

In [122]:
def coin_parametrized_guide1(data):
   
    # bug here with interval(0, 1)
    aa = pyro.param("a", torch.tensor(0.4),
                    constraint=constraints.interval(0, 0.9))
    bb = pyro.param("b", torch.tensor(0.6),
                    constraint=constraints.interval(0, 0.9))
    # print("guide a:", aa)
    
    '''
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0),
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0),
                        constraint=constraints.positive)
    '''
    # print("guide alpha_q:", alpha_q)
    # print("guide beta_q:", beta_q)
    
    res = pyro.sample("coin", dist.Uniform(aa, bb))
    # print("guide: coin", res)
    return res
    # return pyro.sample("coin", dist.Beta(alpha_q, beta_q))

In [123]:
# coin_parametrized_guide1(data)

In [124]:
guess = 8.5
LEARNING_RATE = 1.0e-7

# setup the optimizer:
adam_params = {"lr": 0.05, "betas": (0.9, 0.999)}
optimizer = pyro.optim.Adam(adam_params)

# adam_params = {"lr": 0.00000000000005, "betas": (0.998, 0.999)}
# optimizer = pyro.optim.Adam(adam_args)

# adam_args = {"lr": LEARNING_RATE}
# optimizer = pyro.optim.Adam(adam_args)
# optimizer = pyro.optim.Adam({"lr": 0.00000000001, "betas": (0.95, 0.999)})
# optimizer=pyro.optim.SGD({"lr": 0.01,
#                         "momentum":0.9})

# setup the loss:
# loss = pyro.infer.CSIS()
# loss = pyro.infer.TraceEnum_ELBO()
loss = pyro.infer.Trace_ELBO()
# optimizer.zero_grad()
pyro.clear_param_store()


svi = pyro.infer.SVI(model=conditioned_coin_obs,
                     guide=coin_parametrized_guide1,
                     # guide=coin_parametrized_guide,
                     optim=optimizer,
                     loss=loss)


losses, a,b  = [], [], []
num_steps = 500  # 2500
for t in range(num_steps):
    # import pdb; pdb.set_trace()
    losses.append(svi.step(data))
    a.append(pyro.param("a").item())
    b.append(pyro.param("b").item())
    # print(pyro.param("a").grad)

In [125]:
print(len(a))
# is't last Nan?:
print(a[-1])
# losses

500
0.7339773774147034


In [105]:
import matplotlib.pyplot as plt

In [126]:
%matplotlib
plt.plot(losses)
plt.title("ELBO")
plt.xlabel("step")
plt.ylabel("loss");
print('a = ',pyro.param("a").item())

Using matplotlib backend: Qt5Agg
a =  0.7339773774147034


In [127]:
%matplotlib
plt.subplot(1,2,1)
# plt.plot([0,num_steps],[9.14,9.14], 'k:')
plt.plot(a)
plt.xlabel("step")
plt.ylabel('a')

plt.subplot(1,2,2)
plt.ylabel('b')
# plt.plot([0,num_steps],[0.6,0.6], 'k:')
plt.plot(b)
plt.tight_layout()

Using matplotlib backend: Qt5Agg


In [139]:
p


tensor(0.7340, grad_fn=<AddBackward0>)

In [136]:
p = pyro.param('a')
print(p.item())
p.unbind?

print(a.item())
print(p.item())


0.7339773774147034
0.7339773774147034
0.7339773774147034


##### Appendix:

In [17]:
loss = pyro.infer.Trace_ELBO()
model_trace, guide_trace = loss._get_trace(conditioned_coin_obs,
                                           coin_parametrized_guide,
                                           [data],{})
model_trace.nodes.items()
for name, site in guide_trace.nodes.items():
    if site["type"] == "sample":
        print(name)
        print(site['score_parts'])


coin
ScoreParts(log_prob=tensor(-0.5108, grad_fn=<NegBackward>), score_function=tensor(-0.5108, grad_fn=<NegBackward>), entropy_term=0)


In [18]:
import pyro.poutine as poutine

In [19]:
# strange behaviour of values (bug?):
with poutine.trace(param_only=True) as param_capture:
    
    loss.loss_and_grads(conditioned_coin_obs, coin_parametrized_guide,[data])
params = set(site["value"].unconstrained()
             for site in param_capture.trace.nodes.values())

params

ValueError: Error while computing log_prob at site 'flip':
Input arguments must all be instances of numbers.Number or torch.tensor.
Trace Shapes:     
 Param Sites:     
Sample Sites:     
    coin dist    |
        value    |
     log_prob    |
    flip dist 10 |
        value    |

In [21]:
# leaf tensor?:
adam_params = {"lr": 0.0005, "betas": (0.9, 0.999)}
optimizer = pyro.optim.Adam(adam_params)
optimizer([pyro.param('a')])

ValueError: can't optimize a non-leaf Tensor